# Домашнее задание 3
### Линейная и логистическая регрессия

Оформление дз:
Присылайте выполненное на почту `mlphystech@gmail.com` файлы .pdf и .ipynb или ссылку на свой гитхаб.

**Дедлайн: 11 апреля 23:59**

*Зуева Надежда*

ФИВТ МФТИ

Это задание будет посвящено линейной и логистической регрессии.
Мы надеемся, что вы уже знаете, что такое `kaggle`, так как сегодня нам предстоит поработать с самым *классическим* датасетом машинного обучения, а именно $-$ с Титаником!


Цель соревнования $-$ предсказать, кто из пассажиров выживет, а кто, увы, нет.

https://www.kaggle.com/c/titanic

## Теоретические вопросы
### 2 балла (по 0.5 за каждый верный ответ)
1. Что такое линейные модели классификации и регрессии? В чем их основные свойства? Где хорошо работают?
2. Что такое регуляризация и как она используется? 
3. Что такое MSE? Приведите пример.
4. Что такое градиентный спуск? Опишите базовый алгоритм работы.

P.S. просьба не копипастить из Википедии. Это все легко вычисляется и будет караться :)

## Задание 1 (4 балла)
Скачайте файлы `train.csv` и `test.csv`.

Выведите первые 10 строк из `train.csv`

In [6]:
import pandas
from copy import deepcopy
import numpy as np
import pandas as pd

data = pd.DataFrame(pd.read_csv("/home/maxim/Загрузки/mlschl/Hometasks/Hometask_3/train.csv"))


In [7]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Как мы видим, не все данные *

полны*. Например, это видно в столбце `Age`. 
В качестве простейшего решения мы предлагаем заменить пропущенные значения на медианные (функция `numpy.median`)
Вы можете воспользоваться нашим методом или произвести собственный анализ и заменить значения более умным способом (profit: бонусные баллы).

In [8]:
# Replace all the missing ages in the data with the median age
df = deepcopy(data)
df["Age"] = df["Age"].fillna(df["Age"].median())

Теперь займемся **оцифровкой признаков**

Кстати, напомните, а зачем нам это нужно? :)

Оцифровка позволяет комфортнее работать с данными, потому что когда все данные представлены в одном формате, нет дополнительной головной боли при создании алгоритмов

Какой параметр здесь бинарный? Оцифруйте его при помощи нулей и единиц.

In [9]:
df.loc[df["Sex"] == "male", "Sex"] = 0    #тут что-то должно быть
df.loc[df["Sex"] == "female", "Sex"] = 1 #тут что-то должно быть

Выведите описание нашего датасета после изменений с помощью функции `describe`.
Сколько процентов пассажиров выжило? Каков был средний возраст всех пассажиров?

In [10]:
print (df.describe())

       PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642    0.352413   29.361582   
std     257.353842    0.486592    0.836071    0.477990   13.019697   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000    2.000000    0.000000   22.000000   
50%     446.000000    0.000000    3.000000    0.000000   28.000000   
75%     668.500000    1.000000    3.000000    1.000000   35.000000   
max     891.000000    1.000000    3.000000    1.000000   80.000000   

            SibSp       Parch        Fare  
count  891.000000  891.000000  891.000000  
mean     0.523008    0.381594   32.204208  
std      1.102743    0.806057   49.693429  
min      0.000000    0.000000    0.000000  
25%      0.000000    0.000000    7.910400  
50%      0.000000    0.000000   14.454200  
75%      1.000000    0.000000   31.000000  
max      8.000000

Продолжим заполнять пропуски. Рассмотрим графу пунктов отправления `Embarked` 

Там есть типы данных:

C = Cherbourg, Q = Queenstown, S = Southampton, а также `nan`

In [11]:
print (df["Embarked"].unique())

['S' 'C' 'Q' nan]


In [12]:
import collections
print(collections.Counter(df['Embarked']))

Counter({'S': 644, 'C': 168, 'Q': 77, nan: 2})


С учетом последней ячейки заполните пропущенные значения по своему усмотрению. Объясните свой выбор, затем оцифруйте признак `Embarked` 

Например, заменив все на {0,1,2} или {12,42,10}, как хотите :)

In [13]:
df["Embarked"] = df["Embarked"].fillna("S")
df.loc[df["Embarked"] == "S", "Embarked"] = 0  
df.loc[df["Embarked"] == "C", "Embarked"] = 1
df.loc[df["Embarked"] == "Q", "Embarked"] = 2

Разобьем выборку на `X` и `y`, где для`y` будем строить предсказания

In [14]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [15]:
y = pd.Series(df['Survived'])

In [16]:
X=df.copy()
del X['PassengerId']
del X['Survived']
del X['Name']
del X['Ticket']
del X['Cabin']

Каким образом можно отобрать фичи, которые будут влиять на наш скор? Тут нам помогут ваш жизненный опыт и ` Recursive Feature Elimination` из библиотеки `Sklearn` 

In [21]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [22]:
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)

0.28422308305516214

Проверьте, похожи ли отобранные вами фичи на те, что представленны ниже?

In [ ]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

In [ ]:
kfold = KFold(df.shape[0], n_folds=3, random_state=1)

Выполним предсказания:

In [ ]:
predicts = []
for train, test in kfold:
   #тут ваш код
predicts = np.concatenate(predicts)

#print (predicts)

predicts[predicts > 0.5],predicts[predicts < 0.5] = 1,0


accuracy = np.count_nonzero(df["Survived"] == predicts)/df["Survived"].count()
print ('Скор для линейной регрессии:',accuracy)


## Задание 2 (2 балла)
Теперь попробуем проделать то же самое для **логистической регрессии**

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
model = LogisticRegression(#тут можно подбирать параметры для повышения скора)

Вычислим оценку точности для всех кросс-валидационных фолдов: возвращаем массив баллов из 3-х фолдов

In [ ]:
#тут ваш код

print("Скор для логистической регрессии: " + str(scores.mean()))

###### Как бы вы объяснили прирост качества в логистической регрессии? 
Сможете побить наш бейзлайн? :)

## Задание 3 (1.5 балла)
Отправьте ваше решение в тестирующую систему и посмотрите, какое место вы займете

рассмотрим `test.csv`

In [ ]:
df_test = pandas.read_csv("test.csv")
print(df_test.describe())

Надо вернуть оцифрованные данные к первоначальному виду:

In [ ]:
#тут ваш код по возвращению оцифрованных параметров в первоначальный вид

df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].median())

model.fit(#тут ваш код)
predicts = model.predict(#тут ваш код)

submission = pandas.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": predicts
})

submission.to_csv("submission.csv", index=False) #вот это отправляется в каггл

Можете попробовать и дальше "поиграться" с параметрами, чтобы подняться в лидерборде. Например, попробовать подобрать параметры в регрессорах. 
Удачи!
    Напоминаем, что справку по любой функции (в том числе и по регрессорам) можно получить, если с наведенным курсором на функцию нажать `Tab`+`Shift`